    # Default Perturbation network analysis notebook
    This notebook was automatically generated using freenrgworkflows
    Author: Antonia Mey
    Email: antonia.mey@ed.ac.uk

In [ ]:
    %pylab inline
    import networkanalysis.networkanalysis as n_graph
    import networkanalysis.plotting as n_plot
    import networkanalysis.experiments as n_ex
    import networkanalysis.stats as n_stats
    import networkanalysis
    networkanalysis.__version__

In [ ]:
    # Creating and populating the perturbation network
    pG = n_graph.PerturbationGraph()
    # Change the path below to the csv file containing the individual perturbations
    pG.populate_pert_graph('/Users/toni_brain/Projects/git/freenrgworkflows/tests/io/graph.csv')
    # Uncomment below if you have run multiple runs for some perturbations and add file path
    #pG.add_data_to_graph('/path/to/additional/runs.csv')
    target_compound = pG.compoundList[0] #change this to your target compound
    pG.compute_weighted_avg_paths(target_compound)
    pG.format_free_energies(merge_BM=True,intermed_ID='INT')
    computed_relative_DDGs = pG.freeEnergyInKcal
    print ("Free energies computed from the perturbation network are: ")
    print ("---------------------------------------- ")
    pG.write_free_energies(computed_relative_DDGs)


    ### Experimental data
    It is useful to compare computed free energies to experimental data.
    The cells below will read in your experimental data. Just replace the path to you IC50 data in the
    `IC_50_file` variable 

In [ ]:
    experiments = n_ex.ExperimentalData()
    IC_50_file = 'tests/io/ic50_exp.dat'
    experiments.compute_DDG_from_IC50s(IC_50_file, reference=target_compound)
    experimental_DDGs = experiments.freeEnergiesInKcal
    print ("Free energies computed from IC50 data: ")
    print ("---------------------------------------- ")
    pG.write_free_energies(experimental_DDGs)


    ### Typical plots
    Below a bar plot and scatter plot template for comparing experimental and computed free energy values

In [ ]:
    plotter = n_plot.FreeEnergyPlotter(experimental_DDGs, computed_relative_DDGs)
    ax,fig = plotter.plot_bar_plot(legend=('experimental', 'computed'))

In [ ]:
    plotter.plot_scatter_plot() 


    ### Error analysis on typical statistical measures: R_mean, MUE and Kendall tau_mean
    Below are examples of how to re-sample from the data in order to obtain error bars on correlation coefficients,
    mean unsigned errors and Kendall tau. Returned are confidence intervals of 65% and the median of the distribution.
     However, standard deviations and mean can also
    be returned, though less likely to give good information as these distributions are often heavily skewed and not
    normally distributed. 

In [ ]:
    stats = n_stats.freeEnergyStats()
    stats.generate_statistics(computed_relative_DDGs,experimental_DDGs,repeats=10000)
    r_confidence = stats.R_confidence
    tau_confidence = stats.tau_confidence
    mue_confidence = stats.mue_confidence
    print ("R confidence is: %.2f < %.2f < %.2f" %(r_confidence[1], r_confidence[0], r_confidence[2]))
    print ("Mue confidence is: %.2f < %.2f < %.2f" %(mue_confidence[1], mue_confidence[0], mue_confidence[2]))
    print ("tau confidence is: %.2f < %.2f < %.2f" %(tau_confidence[1], tau_confidence[0], tau_confidence[2]))